# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import os
import numpy as np
import math
import tensorflow as tf

from time import time
from datetime import datetime
from keras.callbacks import TensorBoard

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

from keras.layers.noise import GaussianNoise

import CMAPSAuxFunctions
from CMAPSAuxFunctions import TrainValTensorBoard

from numpy.random import seed
from tensorflow import set_random_seed


Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0.2
l1_lambda_regularization = 0.10
def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_shape=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    
    
#     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
#                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
#                     name='fc3'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models
    model = RULmodel_SN_5(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model




In [6]:
num_features = len(selected_features)

shape = (len(selected_features)*window_size,)
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)


# Load Data

In [7]:
tModel.data_handler.sequence_length = 24
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 129

In [12]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0.7)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0.7
Loading data from file and computing dataframes
cv number
[18, 33, 47, 49, 53, 99, 58, 80, 24, 67, 45, 60, 23, 63, 68, 1, 88, 4, 91, 39, 70, 56, 100, 46, 98, 5, 83, 94, 86, 71, 43, 77, 87, 27, 25, 12, 17, 10, 65, 11, 7, 96, 48, 81, 6, 69, 92, 35, 16, 97, 57, 15, 29, 40, 38, 13, 93, 59, 44, 20, 3, 84, 37, 30, 78, 31, 9, 32, 74, 75]
Printing shapes

Training data (X, y)
(5426, 336)
(5426, 1)
Cross-Validation data (X, y)
(70, 336)
(70, 1)
Testing data (X, y)
(100, 336)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.53424658 -0.34595701 -0.73330414 ... -0.6         0.632
   0.4185254 ]
 [-0.58219178 -0.18167861 -0.65228815 ... -0.6         0.328
   0.69795539]
 [-0.76712329 -0.11668373 -0.39697832 ... -0.6         0.28
   0.63847584]
 [-0.67808219 -0.33009212 -0.5231005  ... -0.4         0.28
   0.50960347]
 [-0.64383562 -0.59211873 -0.24107729 ... -0.8         0.264
 

# Train the model and test some Tunable Model functionalities

In [14]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='auto')

shape = (nFeatures*tModel.data_handler.sequence_length,)


model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')

now = datetime.now()
tensorboard = TrainValTensorBoard(log_dir="./logs2/"+now.strftime("%Y%m%d-%H%M%S")+ "/",write_graph=False)
#,tensorboard = tensorboard
tModel.epochs = 200
tModel.train_model(learningRate_scheduler=lrate, verbose=0,tensorboard = tensorboard)




training with cv
Train on 13267 samples, validate on 30 samples
Epoch 1/200
13267/13267 [==============================] - 0s 31us/step - loss: 8808.4327 - mean_squared_error: 8748.0672 - val_loss: 9281.6611 - val_mean_squared_error: 9221.2275
Logs: dict_items([('val_mean_squared_error', 9221.2275390625), ('val_loss', 9281.6611328125), ('loss', 8808.432718325426), ('mean_squared_error', 8748.067225953022)])
Val_logs: {'mean_squared_error': 9221.2275390625, 'loss': 9281.6611328125}
Epoch 2/200
13267/13267 [==============================] - 0s 8us/step - loss: 7359.3100 - mean_squared_error: 7296.7505 - val_loss: 7342.5625 - val_mean_squared_error: 7277.1748
Logs: dict_items([('val_mean_squared_error', 7277.1748046875), ('val_loss', 7342.5625), ('loss', 7359.309966031201), ('mean_squared_error', 7296.7504834599)])
Val_logs: {'mean_squared_error': 7277.1748046875, 'loss': 7342.5625}
Epoch 3/200
13267/13267 [==============================] - 0s 7us/step - loss: 4406.3735 - mean_squared_err

13267/13267 [==============================] - 0s 9us/step - loss: 334.1840 - mean_squared_error: 245.8059 - val_loss: 1078.0524 - val_mean_squared_error: 990.1014
Logs: dict_items([('val_mean_squared_error', 990.1014404296875), ('val_loss', 1078.0523681640625), ('loss', 334.1840217896842), ('mean_squared_error', 245.80593865169527)])
Val_logs: {'mean_squared_error': 990.1014404296875, 'loss': 1078.0523681640625}
Epoch 40/200
13267/13267 [==============================] - 0s 8us/step - loss: 332.4823 - mean_squared_error: 244.8799 - val_loss: 1073.2682 - val_mean_squared_error: 986.0030
Logs: dict_items([('val_mean_squared_error', 986.0029907226562), ('val_loss', 1073.2681884765625), ('loss', 332.48233080762094), ('mean_squared_error', 244.87993044402862)])
Val_logs: {'mean_squared_error': 986.0029907226562, 'loss': 1073.2681884765625}
Epoch 41/200
13267/13267 [==============================] - 0s 7us/step - loss: 330.7171 - mean_squared_error: 243.7347 - val_loss: 1070.4597 - val_mean

13267/13267 [==============================] - 0s 9us/step - loss: 293.0762 - mean_squared_error: 226.6960 - val_loss: 1005.2621 - val_mean_squared_error: 939.1635
Logs: dict_items([('val_mean_squared_error', 939.1635131835938), ('val_loss', 1005.2620849609375), ('loss', 293.0761513842646), ('mean_squared_error', 226.69600903318474)])
Val_logs: {'mean_squared_error': 939.1635131835938, 'loss': 1005.2620849609375}
Epoch 78/200
13267/13267 [==============================] - 0s 9us/step - loss: 292.4628 - mean_squared_error: 226.5114 - val_loss: 996.3295 - val_mean_squared_error: 930.4608
Logs: dict_items([('val_mean_squared_error', 930.4608154296875), ('val_loss', 996.3294677734375), ('loss', 292.46277770052984), ('mean_squared_error', 226.51142477879574)])
Val_logs: {'mean_squared_error': 930.4608154296875, 'loss': 996.3294677734375}
Epoch 79/200
13267/13267 [==============================] - 0s 9us/step - loss: 292.2917 - mean_squared_error: 226.6596 - val_loss: 1000.1183 - val_mean_sq

13267/13267 [==============================] - 0s 8us/step - loss: 276.7290 - mean_squared_error: 217.3266 - val_loss: 976.3937 - val_mean_squared_error: 917.0095
Logs: dict_items([('val_mean_squared_error', 917.0094604492188), ('val_loss', 976.3937377929688), ('loss', 276.7289867853987), ('mean_squared_error', 217.32664205663716)])
Val_logs: {'mean_squared_error': 917.0094604492188, 'loss': 976.3937377929688}
Epoch 116/200
13267/13267 [==============================] - 0s 9us/step - loss: 276.6330 - mean_squared_error: 217.2573 - val_loss: 977.0886 - val_mean_squared_error: 917.7357
Logs: dict_items([('val_mean_squared_error', 917.7356567382812), ('val_loss', 977.0885620117188), ('loss', 276.6330401224424), ('mean_squared_error', 217.25729545742203)])
Val_logs: {'mean_squared_error': 917.7356567382812, 'loss': 977.0885620117188}
Epoch 117/200
13267/13267 [==============================] - 0s 9us/step - loss: 276.5763 - mean_squared_error: 217.2154 - val_loss: 976.7833 - val_mean_squar

13267/13267 [==============================] - 0s 8us/step - loss: 274.3948 - mean_squared_error: 215.9283 - val_loss: 972.1775 - val_mean_squared_error: 913.6873
Logs: dict_items([('val_mean_squared_error', 913.687255859375), ('val_loss', 972.177490234375), ('loss', 274.39482310956356), ('mean_squared_error', 215.92825255965812)])
Val_logs: {'mean_squared_error': 913.687255859375, 'loss': 972.177490234375}
Epoch 154/200
13267/13267 [==============================] - 0s 9us/step - loss: 274.2745 - mean_squared_error: 215.8287 - val_loss: 974.3260 - val_mean_squared_error: 915.9330
Logs: dict_items([('val_mean_squared_error', 915.9330444335938), ('val_loss', 974.3259887695312), ('loss', 274.2744617055299), ('mean_squared_error', 215.828683869395)])
Val_logs: {'mean_squared_error': 915.9330444335938, 'loss': 974.3259887695312}
Epoch 155/200
13267/13267 [==============================] - 0s 8us/step - loss: 274.2946 - mean_squared_error: 215.8777 - val_loss: 973.6902 - val_mean_squared_er

Logs: dict_items([('val_mean_squared_error', 911.8091430664062), ('val_loss', 969.2484130859375), ('loss', 271.80153191985477), ('mean_squared_error', 214.36841886138973)])
Val_logs: {'mean_squared_error': 911.8091430664062, 'loss': 969.2484130859375}
Epoch 192/200
13267/13267 [==============================] - 0s 8us/step - loss: 271.7869 - mean_squared_error: 214.3758 - val_loss: 968.9941 - val_mean_squared_error: 911.5768
Logs: dict_items([('val_mean_squared_error', 911.5768432617188), ('val_loss', 968.994140625), ('loss', 271.78685056330295), ('mean_squared_error', 214.37581071589827)])
Val_logs: {'mean_squared_error': 911.5768432617188, 'loss': 968.994140625}
Epoch 193/200
13267/13267 [==============================] - 0s 9us/step - loss: 271.5551 - mean_squared_error: 214.1707 - val_loss: 971.2758 - val_mean_squared_error: 913.9393
Logs: dict_items([('val_mean_squared_error', 913.9392700195312), ('val_loss', 971.2757568359375), ('loss', 271.55505930172467), ('mean_squared_error',

In [15]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=True)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_crossVal)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_crossVal)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

70/70 [==============================] - 0s 35us/step
scores
Engine 1, Predicted RUL 81.82805633544922, Rounded RUL 81.0, Real RUL 58.0
Engine 2, Predicted RUL 107.15811920166016, Rounded RUL 107.0, Real RUL 81.0
Engine 3, Predicted RUL 84.05980682373047, Rounded RUL 84.0, Real RUL 57.0
Engine 4, Predicted RUL 5.329617023468018, Rounded RUL 5.0, Real RUL 0.0
Engine 5, Predicted RUL 28.407878875732422, Rounded RUL 28.0, Real RUL 33.0
Engine 6, Predicted RUL 106.09602355957031, Rounded RUL 106.0, Real RUL 122.0
Engine 7, Predicted RUL 125.94464111328125, Rounded RUL 125.0, Real RUL 156.0
Engine 8, Predicted RUL 121.11087036132812, Rounded RUL 121.0, Real RUL 159.0
Engine 9, Predicted RUL 9.373188972473145, Rounded RUL 9.0, Real RUL 19.0
Engine 10, Predicted RUL 84.74337768554688, Rounded RUL 84.0, Real RUL 76.0
Engine 11, Predicted RUL 100.18041229248047, Rounded RUL 100.0, Real RUL 78.0
Engine 12, Predicted RUL 126.4206771850586, Rounded RUL 126.0, Real RUL 123.0
Engine 13, Predicted RU

In [16]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=False)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 40us/step
scores
Engine 1, Predicted RUL 116.12946319580078, Rounded RUL 116.0, Real RUL 112.0
Engine 2, Predicted RUL 124.67621612548828, Rounded RUL 124.0, Real RUL 98.0
Engine 3, Predicted RUL 60.862667083740234, Rounded RUL 60.0, Real RUL 69.0
Engine 4, Predicted RUL 87.47703552246094, Rounded RUL 87.0, Real RUL 82.0
Engine 5, Predicted RUL 94.43400573730469, Rounded RUL 94.0, Real RUL 91.0
Engine 6, Predicted RUL 103.78065490722656, Rounded RUL 103.0, Real RUL 93.0
Engine 7, Predicted RUL 108.21924591064453, Rounded RUL 108.0, Real RUL 91.0
Engine 8, Predicted RUL 88.47247314453125, Rounded RUL 88.0, Real RUL 95.0
Engine 9, Predicted RUL 109.77326965332031, Rounded RUL 109.0, Real RUL 111.0
Engine 10, Predicted RUL 97.78631591796875, Rounded RUL 97.0, Real RUL 96.0
Engine 11, Predicted RUL 88.87660217285156, Rounded RUL 88.0, Real RUL 97.0
Engine 12, Predicted RUL 97.50536346435547, Rounded RUL 97.0, Real RUL 124.0
Engine 13, Predicted